## basic EDA + saving output table

#### modify the 'datapath' below to point to your Volume/csv:

In [0]:
# reading csv from a UC Volume

datapath = f"/Volumes/ogden_demos/bi_workshop/files_uploaded/asset_trend_data.csv"
spark_df = spark.read.csv(datapath, header=True, inferSchema=True) 
display(spark_df)


#### ^^^ in the cell above, next to the 'Table' displayed, hit the + sign to add a 'Data Profile'

#### profiling key metrics by the 'current flag' (using PySpark code):

In [0]:
# Profile the data by 'current flag' (0 vs 1), including Month metrics
from pyspark.sql import functions as F

# List of relevant numeric columns to profile (update as needed)
numeric_cols = [
    'Equity AUC', 'Cash AUC', 'Mutual Fund AUC', 'Other AUC', 'Total AUC'
]

# Aggregate statistics for each 'current flag' value, including Month
profile_df = (
    spark_df.groupBy('current flag')
    .agg(
        F.count('*').alias('count'),
        F.countDistinct('Month').alias('unique_months'),
        F.min('Month').alias('min_month'),
        F.max('Month').alias('max_month'),
        *[F.mean(c).alias(f'{c}_mean') for c in numeric_cols],
        *[F.min(c).alias(f'{c}_min') for c in numeric_cols],
        *[F.max(c).alias(f'{c}_max') for c in numeric_cols]
    )
)

# Format all AUC metrics as currency with no decimals
from pyspark.sql.functions import format_string
auc_stat_cols = [
    f'{c}_mean' for c in numeric_cols
] + [
    f'{c}_min' for c in numeric_cols
] + [
    f'{c}_max' for c in numeric_cols
]

for col_name in auc_stat_cols:
    profile_df = profile_df.withColumn(
        col_name,
        format_string('$%s', F.format_number(F.col(col_name), 0))
    )

display(profile_df)

#### performing some visual data exploration:

In [0]:
# Plot average Total AUC by Month
from pyspark.sql import functions as F

avg_auc_by_month = (
    spark_df.groupBy("Month")
    .agg(F.avg(F.col("Total AUC")).alias("avg_total_auc"))
    .orderBy("Month")
)

# Use the + sign next to 'Table' below, to create an inline Visualization
# (e.g., set 'Month' as key and 'avg_total_auc' as value)

display(avg_auc_by_month)

Databricks visualization. Run in Databricks to view.

#### ^^^ in the cell above, next to the 'Table' displayed, hit the + sign to add a 'Visualization'

#### switching to SQL in the same python notebook...

In [0]:
# Register the DataFrame as a temp view for SQL queries

spark_df.createOrReplaceTempView("asset_trend_temp_view")

#### notice the sql magic commmand, to switch this cell to SQL:

In [0]:
%sql

-- sum all AUC metrics except 'Total AUC'

SELECT 'equity' AS asset_type, SUM(`Equity AUC`) AS auc FROM asset_trend_temp_view
UNION ALL
SELECT 'cash' AS asset_type, SUM(`Cash AUC`) AS auc FROM asset_trend_temp_view
UNION ALL
SELECT 'mutualfund' AS asset_type, SUM(`Mutual Fund AUC`) AS auc FROM asset_trend_temp_view
UNION ALL
SELECT 'other' AS asset_type, SUM(`Other AUC`) AS auc FROM asset_trend_temp_view
;

Databricks visualization. Run in Databricks to view.

#### ^^^ in the cell above, next to the 'Table' displayed, hit the + sign to add a 'Visualization' (e.g. a pie chart)

#### now to save our Bronze data to Unity Catalog...
>> #### (you will need to modify below to point to your catalog.schema)
>> #### (and if multiple users are doing this, you will need to create unique table names or write to different schemas)
>> #### using value characters in columns names (i.e. no blanks)
>> #### and creating some new metrics to add to the table:

#### modify/provide the values (widgets above) for catalog, schema, and table name:

In [0]:
%sql

CREATE OR REPLACE TABLE IDENTIFIER(:catalog || '.' || :schema || '.' || :table) AS
-- above line equivalent to catalog.schema.table designation
-- these are SQL variables that can be desingated in the notebook widgets above

SELECT  Month                                   AS Month,
        `Current Flag`                          AS Current_Flag,
        `Account Service Group`                 AS Account_Service_Group,
        `Account Service Type`                  AS Account_Service_Type,
        `Account Tax Category`                  AS Account_Tax_Category,
        `Branch Market`                         AS Branch_Market,
        `Branch Region`                         AS Branch_Region,
        `Total Accounts`                        AS Total_Accounts,
        `Total AUC`                             AS Total_AUC,
        `Equity AUC`                            AS Equity_AUC,
        `Cash AUC`                              AS Cash_AUC,
        `Mutual Fund AUC`                       AS MutualFund_AUC,
        `Other AUC`                             AS Other_AUC,
        -- new metrics:
        `Total AUC` / `Total Accounts`          AS Total_AUC_per_acct,
        `Equity AUC` / `Total Accounts`         AS Equity_AUC_per_acct,
        `Cash AUC` / `Total Accounts`           AS Cash_AUC_per_acct,
        `Mutual Fund AUC` / `Total Accounts`    AS MutualFund_AUC_per_acct,
        `Other AUC` / `Total Accounts`          AS Other_AUC_per_acct
FROM asset_trend_temp_view;

## Now we head to the Catalog explorer to take a look at our new table...